In [1]:
import pandas as pd
import numpy as np
import math

In [2]:

import operator

In [6]:
#data = pd.read_csv('soyabean-large-data.csv')
data = pd.read_csv('soybean+large/soybean-large.data', names = [i for i in range(36)])
#data.set_index('U', inplace=True)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,diaporthe-stem-canker,6,0,2,1,0,1,1,1,0,...,0,0,0,4,0,0,0,0,0,0
1,diaporthe-stem-canker,4,0,2,1,0,2,0,2,1,...,0,0,0,4,0,0,0,0,0,0
2,diaporthe-stem-canker,3,0,2,1,0,1,0,2,1,...,0,0,0,4,0,0,0,0,0,0
3,diaporthe-stem-canker,3,0,2,1,0,1,0,2,0,...,0,0,0,4,0,0,0,0,0,0
4,diaporthe-stem-canker,6,0,2,1,0,2,0,1,0,...,0,0,0,4,0,0,0,0,0,0


In [ ]:
#data.values

In [43]:
alpha = 0.7

In [8]:
def is_missing(value):
    #if math.isnan(value):
        #return True
    #return False

    if value == '?':
        return True
    return False

In [9]:
def replace_missing_with_set(data):
    dataset = data.to_dict()
    data_missing_replaced = dataset
    for attribute in data.columns:
        values = set(data[attribute])
        set_of_values = []
        for value in values:
            if not is_missing(value):
                set_of_values.append(value)
        for key in dataset[attribute].keys():
            if is_missing(dataset[attribute][key]):
                data_missing_replaced[attribute][key] = set_of_values
    #print X
    return data_missing_replaced

In [10]:
X = replace_missing_with_set(data)
#print X
new_data = pd.DataFrame.from_dict(X)
new_data.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,diaporthe-stem-canker,6,0,2,1,0,1,1,1,0,...,0,0,0,4,0,0,0,0,0,0
1,diaporthe-stem-canker,4,0,2,1,0,2,0,2,1,...,0,0,0,4,0,0,0,0,0,0
2,diaporthe-stem-canker,3,0,2,1,0,1,0,2,1,...,0,0,0,4,0,0,0,0,0,0
3,diaporthe-stem-canker,3,0,2,1,0,1,0,2,0,...,0,0,0,4,0,0,0,0,0,0
4,diaporthe-stem-canker,6,0,2,1,0,2,0,1,0,...,0,0,0,4,0,0,0,0,0,0


In [11]:
#separate_target_attribute
target = new_data[0]
del new_data[0]
new_data.head()

,1,2,3,4,5,6,7,8,9,10,...,26,27,28,29,30,31,32,33,34,35
0,6,0,2,1,0,1,1,1,0,0,...,0,0,0,4,0,0,0,0,0,0
1,4,0,2,1,0,2,0,2,1,1,...,0,0,0,4,0,0,0,0,0,0
2,3,0,2,1,0,1,0,2,1,2,...,0,0,0,4,0,0,0,0,0,0
3,3,0,2,1,0,1,0,2,0,1,...,0,0,0,4,0,0,0,0,0,0
4,6,0,2,1,0,2,0,1,0,2,...,0,0,0,4,0,0,0,0,0,0


In [12]:
#new_data.columns = [i for i in range(1,70)]
#new_data.head()

In [28]:
# similarity matrix
def fuzzy_similarity_relation(data):
    similarity_matrix = {}
    attributes = data.columns
    index = data.index
    c = len(attributes) #no. of columns
    r = len(index)  # no of rows
    for i in range(c):
        similarity_matrix[attributes[i]] = []
        for j in range(r):
            similarity_relation = []
            for k in range(r):
                set_uj = []
                set_uk = []
                if type(data[attributes[i]][index[j]]) == list:
                    set_uj = data[attributes[i]][index[j]]
                else:
                    set_uj.append(data[attributes[i]][index[j]])
                if type(data[attributes[i]][index[k]]) == list:
                    set_uk = data[attributes[i]][index[k]]
                else:
                    set_uk.append(data[attributes[i]][index[k]])
                intersection = 0
                for value in set_uj:
                    if value in set_uk:
                        intersection += 1
                #print j,k,x,y
                similarity_relation.append((intersection)/(math.sqrt(len(set_uj)*len(set_uk))))
            similarity_matrix[attributes[i]].append(similarity_relation)
    return similarity_matrix

In [29]:
XX = fuzzy_similarity_relation(new_data)

In [31]:
type(XX)

dict

In [32]:
similarity_matrix = XX
#similarity_matrix

In [33]:
#tolerance classes
def tolerance_classes(alpha, similarity_matrix):
    X = {}
    for attribute in similarity_matrix:
        X[attribute] = []
        for sim_relation in similarity_matrix[attribute]:
            X[attribute].append([x+1 for x,y in enumerate(sim_relation) if y >= alpha])
    return X

In [34]:
tolerance = tolerance_classes(alpha, similarity_matrix)
#tolerance

In [44]:
#decision classes
def find_target_decision_classes(target):
    decision_variables = set(target.values)
    decision_class = {}
    #group = target.groupby(target.values)
    for i in range(len(target)):
        if decision_class.get(target[i], -1) == -1:
            decision_class[target[i]] = []
        decision_class[target[i]].append(i+1)
    return decision_class

In [45]:
decision_classes = find_target_decision_classes(target)
decision_classes

{'diaporthe-stem-canker': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'charcoal-rot': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 'rhizoctonia-root-rot': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
 'phytophthora-rot': [31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70],
 'brown-stem-rot': [71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90],
 'powdery-mildew': [91, 92, 93, 94, 95, 96, 97, 98, 99, 100],
 'downy-mildew': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
 'brown-spot': [111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  14

In [46]:
len(decision_classes)

19

In [47]:
#lower approximation
def lower_approximation(decision_classes, tolerance):
    gamma = {}
    total = 0
    for d in decision_classes:
        total += len(decision_classes[d])
    for attribute in tolerance:
        pos = set()
        for x,y in enumerate(tolerance[attribute]):
            for d in decision_classes:
                #if all(elem in decision_classes[d] for elem in y):
                if set(y).issubset(set(decision_classes[d])):
                    pos.add(x+1)
        gamma[attribute] = round(len(pos)/(total + 0.0), 2)
    #print gamma
    return gamma

In [ ]:
#lower_approximation(decision_classes, tolerance)

In [ ]:
# prev_gamma = {}
# for attribute in tolerance:
#     prev_gamma[attribute] = 0
# R = set()
# flag = 0
# while 1:
#     new_gamma = lower_approximation(decision_classes, tolerance)
#     new_gamma_sorted = sorted(new_gamma.items(), key=operator.itemgetter(1), reverse = True)
#     T = R.copy()
#     for x in new_gamma_sorted:
#         if x[1] == 1:
#             print x
#             T.add(x[0])
#             flag = 1
#             print "yha"
#             break
#         if x[1] > prev_gamma[x[0]]:
#             T.add(x[0])
#             reduct_set_matrix = similarity_matrix[x[0]]
#             del similarity_matrix[x[0]]
#             new_similarity_matrix = similarity_matrix
#             for attribute in similarity_matrix:
#                 #print attribute
#                 for i in range(len(target)):
#                     for j in range(len(target)):
#                         new_similarity_matrix[attribute][i][j] = min(reduct_set_matrix[i][j], similarity_matrix[attribute][i][j])
#             similarity_matrix = new_similarity_matrix
#             tolerance = tolerance_classes(alpha, similarity_matrix)
#             print x
#             break
#     prev_gamma = new_gamma
#     #print T,R
#     if T == R:
#         print "idhar"
#         break
#     R = T.copy()
#     if flag:
#         break

In [ ]:
# R

In [48]:
def parellel_reduct(prev_gamma, tolerance, R, similarity_matrix):
    new_gamma = lower_approximation(decision_classes, tolerance)
    new_gamma_sorted = sorted(new_gamma.items(), key=operator.itemgetter(1), reverse = True)
    mx = new_gamma_sorted[0][1]
    T = R.copy()
    flag = 0
    for x in new_gamma_sorted:
        if x[1] == mx:
            flag = 1
            if mx == 1:
                T.add(x[0])
                #print 'a', x
                print(T)
                T.remove(x[0])
            elif x[1] > prev_gamma[x[0]]:
                T.add(x[0])
                #print 'b', x
                reduct_set_matrix = similarity_matrix[x[0]]
                del similarity_matrix[x[0]]
                new_similarity_matrix = similarity_matrix
                for attribute in similarity_matrix:
                    for i in range(len(target)):
                        for j in range(len(target)):
                            new_similarity_matrix[attribute][i][j] = min(reduct_set_matrix[i][j], similarity_matrix[attribute][i][j])
                new_tolerance = tolerance_classes(alpha, new_similarity_matrix)
                parellel_reduct(new_gamma, new_tolerance, T.copy(), new_similarity_matrix)
                T.remove(x[0])
                similarity_matrix[x[0]] = reduct_set_matrix
        else:
            break
    if T == R and flag == 0:
        print(R)
        return

In [49]:
prev_gamma = {}
for attribute in tolerance:
    prev_gamma[attribute] = 0
R = set()
parellel_reduct(prev_gamma, tolerance, R, similarity_matrix)

In [ ]:
print "Don't be the same! Be better"

Don't be the same! Be better
